In [1]:
import sys
path_up = '..'
if not path_up in sys.path:
    sys.path.append(path_up)

In [2]:
import os
import re

import pandas as pd
from PySide6.QtCore import QDateTime, QTimeZone, Qt

from funcs.tide import create_qdatetime_and_get_timestamp_ms

### Google Gemini が示したサンプル

In [3]:
date_string = "2025-04-01"
time_string = "09:00:01"

print("--- ローカルタイムゾーンでの処理 (ミリ秒単位タイムスタンプ、int) ---")
timestamp_result_ms = create_qdatetime_and_get_timestamp_ms(date_string, time_string)

if timestamp_result_ms != -1:
    print(f"日付: {date_string}, 時刻: {time_string}")
    print(f"QDateTime のタイムスタンプ (ミリ秒、int): {timestamp_result_ms}")

    # int のタイムスタンプからQDateTimeを再構築して表示
    reconstructed_qdatetime_ms = QDateTime.fromMSecsSinceEpoch(timestamp_result_ms)
    print(f"タイムスタンプから再構築したQDateTime: {reconstructed_qdatetime_ms.toString(Qt.DateFormat.ISODateWithMs)}")
    print(f"再構築したQDateTimeのタイムゾーン: {reconstructed_qdatetime_ms.timeZone().displayName(QTimeZone.StandardTime)}")

print("\n--- UTCでのQDateTime生成の正しい例 (参考、ミリ秒単位タイムスタンプ、int) ---")
# 方法1: ローカルでパース後、UTCに変換
local_qdatetime = QDateTime.fromString(f"{date_string} {time_string}", "yyyy-MM-dd HH:mm:ss")
if local_qdatetime.isValid():
    qdatetime_utc_from_local = local_qdatetime.toTimeZone(QTimeZone.utc())
    print(f"ローカルからUTCに変換したQDateTime: {qdatetime_utc_from_local.toString(Qt.DateFormat.ISODateWithMs)}")
    print(f"そのタイムスタンプ (ミリ秒、int): {qdatetime_utc_from_local.toMSecsSinceEpoch()}")
else:
    print("エラー: ローカルでの日付/時刻文字列のパースに失敗しました。")

# 方法2: QDateTime のコンストラクタで明示的にUTCを指定して生成
from PySide6.QtCore import QDate, QTime, QTimeZone
qdate_obj = QDate.fromString(date_string, "yyyy-MM-dd")
qtime_obj = QTime.fromString(time_string, "HH:mm:ss")

if qdate_obj.isValid() and qtime_obj.isValid():
    # 時刻にミリ秒を追加する場合は、QTimeのコンストラクタを使用します。
    # 今回の入力文字列にはミリ秒がないため、0を設定します。
    qtime_obj_with_ms = QTime(qtime_obj.hour(), qtime_obj.minute(), qtime_obj.second(), 0)

    qdatetime_utc_constructor = QDateTime(qdate_obj, qtime_obj_with_ms, QTimeZone.utc())
    print(f"コンストラクタでUTC指定して生成したQDateTime: {qdatetime_utc_constructor.toString(Qt.DateFormat.ISODateWithMs)}")
    print(f"そのタイムスタンプ (ミリ秒、int): {qdatetime_utc_constructor.toMSecsSinceEpoch()}")
else:
    print("エラー: 日付または時刻オブジェクトの生成に失敗しました。")

--- ローカルタイムゾーンでの処理 (ミリ秒単位タイムスタンプ、int) ---
日付: 2025-04-01, 時刻: 09:00:01
QDateTime のタイムスタンプ (ミリ秒、int): 1743465601000
タイムスタンプから再構築したQDateTime: 2025-04-01T09:00:01.000
再構築したQDateTimeのタイムゾーン: 日本標準時

--- UTCでのQDateTime生成の正しい例 (参考、ミリ秒単位タイムスタンプ、int) ---
ローカルからUTCに変換したQDateTime: 2025-04-01T00:00:01.000+00:00
そのタイムスタンプ (ミリ秒、int): 1743465601000
コンストラクタでUTC指定して生成したQDateTime: 2025-04-01T09:00:01.000+00:00
そのタイムスタンプ (ミリ秒、int): 1743498001000


### VBA (trader) で収集した Excel データを変換

In [4]:
pattern = re.compile(r'trader_([0-9]{4})([0-9]{2})([0-9]{2})\.xlsm')

In [5]:
dir_path_src = "../excel_old"
dir_path_dst = "../excel_test"

files = os.listdir(dir_path_src)
files

['trader_20250516.xlsm',
 'trader_20250515.xlsm',
 'trader_20250519.xlsm',
 'trader_20250513.xlsm',
 'trader_20250522.xlsm',
 'trader_20250514.xlsm',
 'trader_20250520.xlsm',
 'trader_20250523.xlsm',
 'trader_20250521.xlsm']

In [6]:
for name_file in files:
    m = pattern.match(name_file)
    if m:
        str_year = m.group(1)
        str_month = m.group(2)
        str_day = m.group(3)
        date_str = f"{str_year}-{str_month}-{str_day}"
    else:
        date_str = "1970-01-01"
    
    name_excel_src = os.path.join(dir_path_src, name_file)
    body_file = os.path.splitext(os.path.basename(name_file))[0]
    name_excel_dst = os.path.join(dir_path_dst, f"{body_file}.xlsx")
    
    wb = pd.ExcelFile(name_excel_src)
    
    list_ws = wb.sheet_names
    dict_df_out = dict()
    
    for name_ws in list_ws[1:]:
        df = pd.read_excel(wb, name_ws)
        list_ts = [create_qdatetime_and_get_timestamp_ms(date_str, str(t)) for t in df['Time']]
        dict_df_out[name_ws] = pd.DataFrame({'Time': list_ts, 'Price': df['Price']})
    
    with pd.ExcelWriter(name_excel_dst) as writer:
        for name_ws in dict_df_out.keys():
            df_out = dict_df_out[name_ws]
            df_out.to_excel(writer, sheet_name=name_ws, index=False)

print('Completed!')

Completed!
